In [1]:
import requests
import csv
import os # Import os module for path manipulation

# Base URL for CoinMarketCap API
CMC_API_LINK = 'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing'

def fetch_platform_names(limit=5000):
    """
    Fetches cryptocurrency data and returns a sorted list of unique platform names.
    """
    params = {
        'start': 1,
        'limit': limit,
        'sortBy': 'market_cap',
        'sortType': 'desc',
        'convert': 'USD',
        'cryptoType': 'all',
        'tagType': 'all',
        'aux': 'platform' # Only requesting 'platform' in aux for efficiency
    }

    unique_platform_names = set()
    print("Fetching available token platform names from CoinMarketCap API...")

    with requests.Session() as s:
        s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        try:
            res = s.get(CMC_API_LINK, params=params, timeout=30) # Add a timeout
            res.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            data = res.json()

            if data and 'data' in data and 'cryptoCurrencyList' in data['data']:
                for item in data['data']['cryptoCurrencyList']:
                    platform = item.get('platform')
                    if platform and 'name' in platform:
                        unique_platform_names.add(platform['name'])
            else:
                print("Warning: No cryptocurrency data found in the response or unexpected data structure.")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching platform names: {e}")
            print("Please check your internet connection or try again later.")

    return sorted(list(unique_platform_names))

def fetch_and_export_token_data(selected_platform_name):
    """
    Fetches cryptocurrencies for a given platform name, prints them, and exports them to a CSV.
    """
    params = {
        'start': 1,
        'limit': 500,  # Limit for fetching specific platform tokens
        'sortBy': 'market_cap',
        'sortType': 'desc',
        'convert': 'USD,BTC,ETH',
        'cryptoType': 'all',
        'tagType': 'all',
        'audited': 'false',
        'aux': 'ath,atl,high24h,low24h,num_market_pairs,cmc_rank,date_added,tags,platform,max_supply,circulating_supply,total_supply,volume_7d,volume_30d'
    }

    # Define the CSV file name dynamically based on the selected platform
    # Sanitize the name for filename usage
    safe_platform_name = selected_platform_name.replace(" ", "_").replace("/", "_").replace("\\", "_").lower()
    csv_filename = f"{safe_platform_name}_tokens.csv"

    # Define the headers for the CSV file
    fieldnames = ['CMC Rank', f'Coin Name ({selected_platform_name}-based)', 'Token Mint Address']

    tokens_data = []

    print(f"\nFetching '{selected_platform_name}'-based cryptocurrencies...")

    with requests.Session() as s:
        s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
        try:
            res = s.get(CMC_API_LINK, params=params, timeout=30) # Add a timeout
            res.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
            data = res.json()

            if data and 'data' in data and 'cryptoCurrencyList' in data['data']:
                print(f"\n--- Live Output for {selected_platform_name} Tokens ---")
                print(f"{'CMC Rank':<9} | {'Coin Name':<28} | {'Token Mint Address'}")
                print(f"--------------------------------------------------------------------------------")
                for item in data['data']['cryptoCurrencyList']:
                    platform = item.get('platform')

                    if platform and platform.get('name') == selected_platform_name:
                        rank = item['cmcRank']
                        coin_name = item['name']
                        token_mint_address = platform.get('token_address') or platform.get('contract_address')

                        # Determine the address to print and store
                        display_address = token_mint_address if token_mint_address else 'N/A (Not Available via CMC API)'

                        # Print to console
                        print(f"{rank:<9} | {coin_name:<28} | {display_address}")

                        # Add data to our list for CSV export
                        tokens_data.append({
                            'CMC Rank': rank,
                            f'Coin Name ({selected_platform_name}-based)': coin_name,
                            'Token Mint Address': display_address # Use the determined address here
                        })
                print(f"--------------------------------------------------------------------------------")
            else:
                print("No cryptocurrency data found in the response for this platform or unexpected data structure.")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching token data for '{selected_platform_name}': {e}")
            print("Please check your internet connection or try again later.")
            return # Exit function if data fetch fails

    # Write the collected data to a CSV file
    if tokens_data:
        try:
            with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(tokens_data)
            print(f"\nSuccessfully exported {len(tokens_data)} '{selected_platform_name}'-based cryptocurrencies to '{csv_filename}'")
            print(f"File saved in: {os.path.abspath(csv_filename)}")
        except IOError:
            print(f"I/O error: Could not write to file '{csv_filename}'")
    else:
        print(f"No '{selected_platform_name}'-based cryptocurrencies found to export.")

def main():
    """
    Main function to orchestrate the script's execution.
    """
    # 1. Start the script by listing the available token names
    available_platforms = fetch_platform_names()

    if not available_platforms:
        print("Could not retrieve available platform names. Exiting.")
        return

    print("\nAvailable Token Platform Names:")
    for i, name in enumerate(available_platforms):
        print(f"  {i+1}. {name}")

    # 2. Provide an input command so we can write the token name
    selected_platform = None
    while selected_platform is None:
        user_input = input("\nEnter the exact name of the token platform you want to query (or 'exit' to quit): ").strip()
        if user_input.lower() == 'exit':
            print("Exiting script.")
            return

        if user_input in available_platforms:
            selected_platform = user_input
        else:
            print(f"'{user_input}' is not a valid platform name. Please choose from the list above or enter 'exit'.")

    # 3. After selecting our token, we will trigger the subroutine
    fetch_and_export_token_data(selected_platform)

if __name__ == "__main__":
    main()

Fetching available token platform names from CoinMarketCap API...

Available Token Platform Names:
  1. Algorand
  2. Aptos
  3. Arbitrum
  4. Arbitrum Nova
  5. Astar
  6. Atomicals-ARC20
  7. Avalanche C-Chain
  8. Avalanche DFK
  9. BNB Beacon Chain (BEP2)
  10. BNB Smart Chain (BEP20)
  11. Base
  12. Bera Chain
  13. Bitcichain
  14. Bitcoin
  15. Blast
  16. Canto
  17. Cardano
  18. Celo
  19. Chiliz Chain
  20. Chiliz Legacy Chain
  21. Core
  22. Cosmos
  23. Cronos
  24. Cronos zkEVM Mainnet
  25. Dogechain(EVM)
  26. EOS
  27. Ethereum
  28. Etherlink
  29. Fantom
  30. Flare Network
  31. Fuse
  32. Gnosis Chain
  33. HECO
  34. Hedera Hashgraph
  35. Hyperliquid
  36. ICON
  37. ICP
  38. Injective
  39. KAIA
  40. Linea
  41. Manta Pacific
  42. Mantle
  43. Merlin
  44. Metal
  45. Metis Andromeda
  46. Moonbeam
  47. MultiversX
  48. Near
  49. Neo
  50. ONT
  51. ONUS
  52. Optimism
  53. Ordinals - BRC20
  54. Osmosis
  55. Polkadot
  56. Polygon
  57. PulseChain
  58